In [3]:
import pandas as pd

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Reference File
# Date File
dt_df = pd.read_csv('data/date.csv')
dt_df['from'] = pd.to_datetime(dt_df['from'])
dt_df['from_year'] = dt_df['from'].apply(lambda x: x.year)
dt_df['from_month'] = dt_df['from'].apply(lambda x: x.month)
dt_df['to'] = pd.to_datetime(dt_df['to'])
dt_df['to_year'] = dt_df['from'].apply(lambda x: x.year)
dt_df['to_month'] = dt_df['from'].apply(lambda x: x.month)

# Object File
obj_df = pd.read_csv('data/object.csv')
obj_df = obj_df.drop_duplicates()[['course_id', 'module_id', 'category', 'start']]
#obj_df['start'] = obj_df[obj_df['start'] != 'null']['start'].astype('datetime64')
obj_df['start'] = pd.to_datetime(obj_df[obj_df['start'] != 'null']['start'])
sample_sub_df = pd.read_csv('data/sampleSubmission.csv', header=None)

# Train
tr_enroll = pd.read_csv('data/train/enrollment_train.csv')
tr_log = pd.read_csv('data/train/log_train.csv')
tr_log['time'] = pd.to_datetime(tr_log['time'])
tr_log['action_date'] = tr_log.time.apply(lambda x: x.date())
tr_log['action_dow'] = tr_log['time'].apply(lambda x: x.weekday())
tr_truth = pd.read_csv('data/train/truth_train.csv', header=None)

TypeError: parser_f() got an unexpected keyword argument 'index'

In [4]:
"""
Join All Data files for Training
"""
print tr_log.shape
big_df = pd.merge(tr_enroll, tr_log, on='enrollment_id', how='left')
big_df = pd.merge(big_df, dt_df, on='course_id', how='left')
big_df = pd.merge(big_df, obj_df, left_on=['course_id', 'object'], right_on=['course_id', 'module_id'], how='left')
print big_df.shape
big_df['before_end'] = (big_df['to'] - big_df['time']).astype('m8[D]')
big_df['after_start'] = (big_df['time'] - big_df['from']).astype('m8[D]')
big_df['class_len'] = (big_df['to'] - big_df['from']).astype('m8[D]')
big_df['after_module_released'] = (big_df['time'] - big_df['start']).astype('m8[D]')

(8157277, 7)
(8157277, 18)


In [5]:
user_df = pd.merge(tr_enroll, tr_log, on='enrollment_id', how='left')
user_course_grp = user_df.groupby(['username', 'course_id'])

In [6]:
user_course_df = pd.DataFrame(user_course_grp.event.count())
course_grped = pd.merge(tr_enroll, tr_log, on='enrollment_id', how='left').groupby('course_id')

In [29]:
tr_truth = pd.read_csv('data/train/truth_train.csv', header=None)

ndf = pd.DataFrame()
ndf['enrollment_id'] = tr_enroll['enrollment_id']
ndf['username'] = tr_enroll['username']
user_drop_rate = pd.DataFrame(tr_enroll.join(tr_truth[1]).groupby('username')[1].sum()/ tr_enroll.join(tr_truth[1]).groupby('username')[1].count()).reset_index()

user_drop_rate.columns = ['username', 'user_drop_rate']
pd.merge(ndf, user_drop_rate, on='username', how='left').head()

,enrollment_id,username,user_drop_rate
0,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,0.333333
1,3,1qXC7Fjbwp66GPQc6pHLfEuO8WKozxG4,0.000000
2,4,FIHlppZyoq8muPbdVxS44gfvceX9zvU7,0.500000
3,5,p1Mp7WkVfzUijX0peVQKSHbgd5pXyl4c,0.000000
4,6,dpK33RH9yepUAnyoywRwBt1AJzxGlaja,0.000000


In [38]:
tr_truth = pd.read_csv('data/train/truth_train.csv', header=None)

ndf = pd.DataFrame()
ndf['enrollment_id'] = tr_enroll['enrollment_id']
ndf['course_id'] = tr_enroll['course_id']
course_drop_rate = pd.DataFrame(tr_enroll.join(tr_truth[1]).groupby('course_id')[1].sum()/ tr_enroll.join(tr_truth[1]).groupby('course_id')[1].count()).reset_index()

course_drop_rate.columns = ['course_id', 'course_drop_rate']
pd.merge(ndf, course_drop_rate, on='course_id', how='left')

,enrollment_id,course_id,course_drop_rate
0,1,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,0.696183
1,3,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,0.821067
2,4,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,0.696183
3,5,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,0.821067
4,6,AXUJZGmZ0xaYSWazu8RQ1G5c76ECT1Kd,0.833312
5,7,7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx,0.821067
6,9,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,0.696183
7,12,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,0.696183
8,13,5X6FeZozNMgE2VRi3MJYjkkFK8SETtu2,0.885301
9,14,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,0.696183


In [84]:
"""
Course Level Features
"""
def avg_activity_per_course(enr_log_df):
    """
    Average Activity Per Course Per Enrollment
    
    Key: course_id
    """
    user_course_grp = enr_log_df.groupby(['username', 'course_id'])
    return user_course_grp.event.count().reset_index().groupby('course_id').event.mean()
avg_activity_per_course(user_df)

event_list = user_df.event.unique().tolist()
def avg_activity_per_course_event(enr_log_df, event_name):
    """
    Average Activity Per Course-Event Per Enrollment
    """
    user_course_grp = enr_log_df[enr_log_df['event']==event_name].groupby(['username', 'course_id'])
    return user_course_grp.event.count().reset_index().groupby('course_id').event.mean()
# for event_name in event_list:
#     print avg_activity_per_course_event(user_df, event_name)

In [85]:
avg_activity_per_course(user_df)

course_id
1pvLqtotBsKv7QSOsLicJDQMHx3lui6d     87.270903
3VkHkmOtom3jM2wCu94xgzzu1d6Dn7or     77.479582
3cnZpv6ReApmCaZyaQwi2izDZxVRdC01     85.456275
5Gyp41oLVo7Gg7vF4vpmggWP5MU70QO6     49.478275
5X6FeZozNMgE2VRi3MJYjkkFK8SETtu2     38.962138
7GRhBDsirIGkRZBtSMEzNTyDr2JQm4xx    100.270088
81UZtt1JJwBFYMj5u38WNKCSVA4IJSDv     54.373625
9Bd26pfDLvkPINwLnpaGcf0LrLUvY1Mz     92.661249
9Mq1P5hrrLw6Bh9X4W4ZjisQJDdxjz9x     34.454032
9zpXzW9zCfU8KGBWkhlsGH8B8czISH4J     14.422842
A3fsA9Zfv1X2fVEQhTw51lKENdNrEqT3     56.301895
AXUJZGmZ0xaYSWazu8RQ1G5c76ECT1Kd     43.077428
DABrJ6O4AotFwuAbfo1fuMj40VmMpPGX     62.286585
DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila     85.072563
Er0RFawC4sHagDmmQZcBGBrzamLQcblZ    107.947932
G8EPVSXsOYB5YQWZGiz1aVq5Pgr2GrQu     71.750632
H2lDW05SyKnwntZ6Fora76aPAEswcMa5     55.689165
HbeAZjZFFQUe90oTP0RRO0PEtRAqU3kK    113.378138
I7Go4XwWgpjRJM8EZGEnBpkfSmBNOlsO     96.687060
KHPw0gmg1Ad3V07TqRpyBzA8mRjj7mkt     28.331143
NmbZ3BmS8V4pMg6oxXHWpqqMZCE1jvYt     36.771048
RXD

In [ ]:
"""
Course Level Features

(optional)
- Course Drop Rate
- Personal Historical Drop Rate

(Clustering Course)
- Cluster Drop Rate

(Cluster Person)
(by ......)
- Cluster Drop Rate

"""

In [ ]:
"""
Enrollment Level Features

- Total # of Activity

- Total # of problem
- Total # of video
- Total # of access
- Total # of wiki
- Total # of discussion
- Total # of navigate
- Total # of page close

- Perct of problem
- Perct of video
- Perct of access
- Perct of wiki
- Perct of discussion
- Perct of navigate
- Perct of page close

- total problem - course level avg
- total vedio - course level avg
- total access - course level avg
- total wiki - course level avg
- total discussion - course level avg
- total navigate - course level avg
- total page close - course level avg

- percentile of # of Activity
- percentile of # of problem
- percentile of # of vedio
- percentile of # of access
- percentile of # of wiki
- percentile of # of discussion
- percentile of # of navigate
- percentile of # of page close

- Total # of unique activity day

- Most activity DOW
- 2nd most activity DOW
- 3rd most activity DOW

TODO:
[filter: First 15 Days]
All Above

[filter: Last 15 Days]
All Above

Difference Begin and End:

- Slope of # of activity



"""

In [ ]:
"""
User Level Feature
- # of dropped / # of enrollment

"""

In [ ]:
"""
Investigation
- How many people in test is also in train
25102.0/80362 = .312 (30% of people in test are also in train)

- How many people in test is also in train has dropped
21791.0/80362 = 0.271

- Course overlap between train and test
Test: 39 Courses
Train: 39 Courses
100% Overlap
"""

In [90]:
big_df.head()

,enrollment_id,username,course_id,time,source,event,object,action_date,action_dow,from,...,from_month,to_year,to_month,module_id,category,start,before_end,after_start,class_len,after_module_released
0,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:29,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b,2014-06-14,5,2014-06-12,...,6,2014,6,NaN,NaN,NaT,26,2,29,NaN
1,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:39,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,2014-06-14,5,2014-06-12,...,6,2014,6,NaN,NaN,NaT,26,2,29,NaN
2,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:39,server,access,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4,2014-06-14,5,2014-06-12,...,6,2014,6,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4,sequential,2013-11-04 01:00:00,26,2,29,222
3,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:48,server,access,2cmZrZW2h6Il91itO3e89FGcABLWhf3W,2014-06-14,5,2014-06-12,...,6,2014,6,2cmZrZW2h6Il91itO3e89FGcABLWhf3W,sequential,2014-06-12 16:00:00,26,2,29,1
4,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:41:49,browser,problem,RMtgC2bTAqEeftenUUyia504wsyzeZWf,2014-06-14,5,2014-06-12,...,6,2014,6,RMtgC2bTAqEeftenUUyia504wsyzeZWf,problem,NaT,26,2,29,NaN


In [115]:
train_base = big_df.groupby(['enrollment_id', 'course_id', 'from_year', 'from_month', 'to_year', 'to_month']).agg({
    'event': 'count',
    'action_dow': 'median',
    'before_end': 'median',
    'after_start': 'median'
}).reset_index()

In [98]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from functools import partial
import sklearn.metrics as metrics
from sklearn import cross_validation
from sklearn import ensemble, preprocessing
from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from geopy.distance import vincenty, great_circle
from pylab import *
%matplotlib inline

In [116]:
print train_base.shape
labels = tr_truth[1]
print labels.shape
train_base

(120542, 9)
(120542,)


,enrollment_id,from_year,from_month,to_year,to_month,before_end,after_start,event,action_dow
0,1,2014,6,2014,6,8.0,20.0,314,4
1,3,2014,6,2014,6,13.0,15.0,288,3
2,4,2014,6,2014,6,17.0,11.0,99,2
3,5,2014,6,2014,6,17.0,11.0,633,4
4,6,2014,6,2014,6,0.0,28.0,23,2
5,7,2014,6,2014,6,15.0,13.0,479,3
6,9,2014,6,2014,6,4.0,24.0,97,1
7,12,2014,6,2014,6,8.0,20.0,127,3
8,13,2014,6,2014,6,4.0,24.0,463,3
9,14,2014,6,2014,6,8.5,19.5,102,2


In [117]:
# Random Forest Classifier
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1)
cv = cross_validation.ShuffleSplit(train_base.shape[0], n_iter=5,
        test_size=0.3, random_state=0)
scores = cross_validation.cross_val_score(
            clf, train_base, labels, cv=cv)
print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1)
cv = cross_validation.ShuffleSplit(train_base.shape[0], n_iter=10,
        test_size=0.3, random_state=0)
scores = cross_validation.cross_val_score(
            clf, train_base, labels, cv=cv, scoring='roc_auc')
print("ROC: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.84650 (+/- 0.00)
ROC: 0.83994 (+/- 0.00)


In [121]:
# Test
te_enroll = pd.read_csv('data/test/enrollment_test.csv')
te_log = pd.read_csv('data/test/log_test.csv')
te_log['time'] = pd.to_datetime(te_log['time'])
te_log['action_date'] = te_log.time.apply(lambda x: x.date())
te_log['action_dow'] = te_log['time'].apply(lambda x: x.weekday())


In [122]:
"""
Join All Data files for Training
"""
te_big_df = pd.merge(te_enroll, te_log, on='enrollment_id', how='left')
te_big_df = pd.merge(te_big_df, dt_df, on='course_id', how='left')
te_big_df = pd.merge(te_big_df, obj_df, left_on=['course_id', 'object'], right_on=['course_id', 'module_id'], how='left')
print big_df.shape
te_big_df['before_end'] = (te_big_df['to'] - te_big_df['time']).astype('m8[D]')
te_big_df['after_start'] = (te_big_df['time'] - te_big_df['from']).astype('m8[D]')
te_big_df['class_len'] = (te_big_df['to'] - te_big_df['from']).astype('m8[D]')
te_big_df['after_module_released'] = (te_big_df['time'] - te_big_df['start']).astype('m8[D]')

(8157277, 22)


In [135]:
test_base = te_big_df.groupby(['enrollment_id', 'from_year', 'from_month', 'to_year', 'to_month']).agg({
    'event': 'count',
    'action_dow': 'median',
    'before_end': 'median',
    'after_start': 'median'
}).reset_index()

In [133]:
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1)
clf.fit(train_base, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=1,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [136]:
prediction = clf.predict(test_base)

In [141]:
sample_sub_df[1] = prediction

In [7]:
ndf = pd.DataFrame()
big_df = pd.merge(tr_enroll, tr_log, on='enrollment_id', how='left')
big_df = pd.merge(big_df, dt_df, on='course_id', how='left')
big_df['before_end'] = (big_df['to'] - big_df['time']).astype('m8[D]')
big_df['after_start'] = (big_df['time'] - big_df['from']).astype('m8[D]')
big_df['class_len'] = (big_df['to'] - big_df['from']).astype('m8[D]')    

In [8]:
big_df.head()

,enrollment_id,username,course_id,time,source,event,object,action_date,action_dow,from,to,from_year,from_month,to_year,to_month,before_end,after_start,class_len
0,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:29,server,navigate,Oj6eQgzrdqBMlaCtaq1IkY6zruSrb71b,2014-06-14,5,2014-06-12,2014-07-11,2014,6,2014,6,26,2,29
1,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:39,server,access,3T6XwoiMKgol57cm29Rjy8FXVFcIomxl,2014-06-14,5,2014-06-12,2014-07-11,2014,6,2014,6,26,2,29
2,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:39,server,access,qxvBNYTfiRkNcCvM0hcGwG6hvHdQwnd4,2014-06-14,5,2014-06-12,2014-07-11,2014,6,2014,6,26,2,29
3,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:38:48,server,access,2cmZrZW2h6Il91itO3e89FGcABLWhf3W,2014-06-14,5,2014-06-12,2014-07-11,2014,6,2014,6,26,2,29
4,1,9Uee7oEuuMmgPx2IzPfFkWgkHZyPbWr0,DPnLzkJJqOOPRJfBxIHbQEERiYHu5ila,2014-06-14 09:41:49,browser,problem,RMtgC2bTAqEeftenUUyia504wsyzeZWf,2014-06-14,5,2014-06-12,2014-07-11,2014,6,2014,6,26,2,29


In [14]:
ndf = pd.DataFrame()
ndf['enrollment_id'] = big_df['enrollment_id']
ndf = ndf.join(pd.core.reshape.get_dummies(big_df['after_start'].values.tolist()))
user_cluster_df = ndf.groupby('enrollment_id').sum()

In [24]:
#tr_truth


KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [46]:
from sklearn.cluster import KMeans
ndf = pd.DataFrame()
ndf['enrollment_id'] = big_df['enrollment_id']
ndf = ndf.join(pd.core.reshape.get_dummies(big_df['after_start'].values.tolist()))
user_cluster_df = ndf.groupby('enrollment_id').sum()
for i in range(4,6):
    k_means = KMeans(init='k-means++', n_clusters=i, n_init=20)
    k_means.fit(user_cluster_df.values)
    k_means_labels = k_means.labels_
    k_means_cluster_centers = k_means.cluster_centers_
    k_means_labels_unique = np.unique(k_means_labels)
    ndf = pd.DataFrame()
    ndf['enrollment_id'] = tr_enroll['enrollment_id']
    ndf['kmean_labels'] = k_means_labels
    tr_truth = pd.read_csv('data/train/truth_train.csv', header=None)
    ndf['truth'] = tr_truth[1]
    print ndf.groupby('kmean_labels').truth.sum()/ndf.groupby('kmean_labels').truth.count()
    print ndf.groupby('kmean_labels').truth.count()

kmean_labels
0    1
1    0
2    0
3    1
Name: truth, dtype: float64
kmean_labels
0    90366
1    13267
2    11694
3     5215
Name: truth, dtype: int64
kmean_labels
0    1
1    0
2    0
3    1
4    0
Name: truth, dtype: float64
kmean_labels
0    90366
1    10601
2    13267
3     5215
4     1093
Name: truth, dtype: int64


In [30]:
ndf = pd.DataFrame()
ndf['enrollment_id'] = tr_enroll['enrollment_id']
ndf['kmean_labels'] = k_means_labels
tr_truth = pd.read_csv('data/train/truth_train.csv', header=None)
ndf['truth'] = tr_truth[1]
ndf.groupby('kmean_labels').truth.sum()

In [47]:
ndf

,enrollment_id,kmean_labels,truth
0,1,1,0
1,3,1,0
2,4,2,0
3,5,1,0
4,6,2,0
5,7,3,1
6,9,0,1
7,12,2,0
8,13,1,0
9,14,0,1


In [49]:
k_means.predict(user_cluster_df.values)

array([1, 1, 2, ..., 0, 0, 0], dtype=int32)